## <span style="color: skyblue; ">7.2 パターンマッチング・コールバック</span>

- パターンマッチング・コールバック
  - 条件にあったコンポーネントの値を活用するコールバック
  - UI に新たに加えたコンポーネントに対して、コールバックを適用できる
- パターンマッチング・コールバックを作成するには
  - コンポーネントの ID を辞書型で与えて
  - Dash.callback デコレータ内でセレクタを用いて条件を指定する
  - セレクタ
    - ALL セレクタ
    - MATCH セレクタ
    - ALLSMALLER セレクタ
    - Dash．callback デコレータに渡す Output クラスなどの ID の辞書に格納する
- [Pattern-Matching Callbacks](https://dash.plotly.com/pattern-matching-callbacks)

### <span style="color: skyblue; ">7.2.1 ALL セレクタ</span>

- ALL セレクタ
  - ID キー`"type"`が一致するすべてのコンポーネントをコールバックで利用できるセレクタ

In [ ]:
# gapminder データセットを用いて、ボタンをクリックすると新たにドロップダウンが追加されるアプリケーション
# ドロップダウンでは country 列に含まれる国名が選択できる
# ドロップダウンで選択されたすべての国名が出力される

from jupyter_dash import JupyterDash
from dash import html, dcc
from dash.dependencies import Input, Output, State, ALL
import plotly.express as px

gapminder = px.data.gapminder()

app = JupyterDash(__name__)

# レイアウト
def server_layout():
    return html.Div([
        html.Button("PUSH ME", id="add-drop"),
        html.Div(id="show-drop", children=[]),
        html.P(id="selected-text"),
    ], style=dict(width="80%", margin="2% auto"),)
app.layout = server_layout

# コールバック1
@app.callback(
    Output(component_id="show-drop", component_property="children"),
    Input(component_id="add-drop", component_property="n_clicks"),
    State(component_id="show-drop", component_property="children"),
    # アプリケーション起動時にコールバックが動作しない
    prevent_initial_call=True,
)
def update_layout(n_clicks, children):
    new_layout = html.Div([
        dcc.Dropdown(
            id=dict(type="add-dropdown", index=n_clicks),
            options=[
                dict(label=c, value=c) for c in gapminder.country.unique()
            ],
            value=gapminder.country.unique()[n_clicks - 1],
        )
    ])
    children.append(new_layout)
    return children
# コールバック2
@app.callback(
    Output(component_id="selected-text", component_property="children"),
    Input(component_id=dict(type="add-dropdown", index=ALL), component_property="value"),
    prevent_initial_call=True,
)
def update_graph(selected_values):
    return str(selected_values)

if __name__ == "__main__":
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


In [ ]:
# ドロップダウンで選択された値を
# 折れ線グラフに表示する

from jupyter_dash import JupyterDash
from dash import html, dcc
from dash.dependencies import Input, Output, State, ALL
import plotly.express as px

gapminder = px.data.gapminder()

app = JupyterDash(__name__)

# レイアウト
def server_layout():
    return html.Div([
        html.Button("PUSH ME", id="input-button"),
        html.Div(id="show-drop", children=[]),
        html.Div(id="show-graph"),
    ])
app.layout = server_layout

# コールバック1
# ドロップダウンを追加する
@app.callback(
    Output(component_id="show-drop", component_property="children"),
    Input(component_id="input-button", component_property="n_clicks"),
    State(component_id="show-drop", component_property="children"),
    # アプリケーション起動時にコールバックが動作しない
    prevent_initial_call=True,
)
def update_layout(n_clicks, children):
    new_layout = html.Div([
        dcc.Dropdown(
            id=dict(type="add-dropdown", index=n_clicks),
            options=[
                dict(label=c, value=c) for c in gapminder.country.unique()
            ],
            value=gapminder.country.unique()[n_clicks - 1],
        )
    ])
    children.append(new_layout)
    return children
# コールバック2
# ドロップダウンに表示された値をすべて折れ線グラに表示する
@app.callback(
    Output(component_id="show-graph", component_property="children"),
    Input(component_id=dict(type="add-dropdown", index=ALL), component_property="value"),
    prevent_initial_call=True,
)
def update_graph(selected_values):
    # 1つの折れ線グラフに表示する
    selected_countries = gapminder[gapminder["country"].isin(selected_values)]
    return dcc.Graph(
        figure=px.line(
            selected_countries, x="year", y="lifeExp", color="country"
        )
    )

if __name__ == "__main__":
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


### <span style="color: skyblue; ">7.2.2 MATCH セレクタ</span>

- MATCH セレクタ
  - ID キー`"index"`の値が一致するコンポーネントに対して、コールバックを適用する

In [ ]:
# gapminder データセットを用いて、ボタンをクリックすると新たにドロップダウンが追加されるアプリケーション
# ドロップダウンでは country 列に含まれる国名が選択できる
# ドロップダウンで選択された国名が出力される

from jupyter_dash import JupyterDash
from dash import html, dcc
from dash.dependencies import Input, Output, State, MATCH
import plotly.express as px

gapminder = px.data.gapminder()

app = JupyterDash(__name__)

# レイアウト
def server_layout():
    return html.Div([
        html.Button("PUSH ME", id="add-drop"),
        html.Div(id="show-drop", children=[]),
    ], style=dict(width="80%", margin="2% auto"),)
app.layout = server_layout

# コールバック1
# ボタンを押すたびに
# UI（「ドロップダウン」と「P要素」）が追加される
@app.callback(
    Output(component_id="show-drop", component_property="children"),
    Input(component_id="add-drop", component_property="n_clicks"),
    State(component_id="show-drop", component_property="children"),
    # アプリケーション起動時にコールバックが動作しない
    prevent_initial_call=True,
)
def update_layout(n_clicks, children):
    new_layout = html.Div([
        dcc.Dropdown(
            id=dict(type="add-dropdown", index=n_clicks),
            options=[
                dict(label=c, value=c) for c in gapminder.country.unique()
            ],
            value=gapminder.country.unique()[n_clicks - 1],
        ),
        html.P(id=dict(type="show-text", index=n_clicks)),
    ])
    children.append(new_layout)
    return children

# コールバック2
# UIが追加されたら、
# 選択された値をID=show-textのP要素に値を渡す
@app.callback(
    Output(component_id=dict(type="show-text", index=MATCH), component_property="children"),
    Input(component_id=dict(type="add-dropdown", index=MATCH), component_property="value"),
    prevent_initial_call=False,
)
def update_graph(selected_values):
    return str(selected_values)

if __name__ == "__main__":
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


In [ ]:
# MATCHセレクタを用いて
# 2つのドロップダウンとグラフが加えられて、
# ドロップダウンで選択した国名と項目をグラフに表示するアプリケーション

from jupyter_dash import JupyterDash
from dash import html, dcc
from dash.dependencies import Input, Output, State, MATCH
import plotly.express as px

gapminder = px.data.gapminder()

app = JupyterDash(__name__)

# レイアウト
def server_layout():
    return html.Div([
        html.Button("PUSH ME", id="add-drop", n_clicks=0),
        html.Div(id="show-drop", children=[]),
    ])
app.layout = server_layout

# コールバック1
# ボタンを押すたびに
# UI（「ドロップダウン」と「グラフエリア」）が追加される
@app.callback(
    Output(component_id="show-drop", component_property="children"),
    Input(component_id="add-drop", component_property="n_clicks"),
    State(component_id="show-drop", component_property="children"),
    # アプリケーション起動時にコールバックが動作しない
    prevent_initial_call=True,
)
def update_layout(n_clicks, children):
    new_layout = html.Div(
        [
            # 国名表示
            dcc.Dropdown(
                id=dict(type="add-dropdown", index=n_clicks),
                options=[
                    dict(label=c, value=c) for c in gapminder.country.unique()
                ],
                value=gapminder.country.unique()[n_clicks - 1],
            ),
            # 項目表示
            dcc.Dropdown(
                id=dict(type="add-dropdown2", index=n_clicks),
                options=[
                    dict(label=col, value=col) for col in gapminder.columns[3:6]
                ],
                value="lifeExp",
            ),
            # グラフ表示
            dcc.Graph(id=dict(type="show-graph", index=n_clicks)),
        ],
        style=dict(width="50%", display="inline-block")
    )
    children.append(new_layout)
    return children

# コールバック2
# UIが追加されたら、
# 選択された値をID=show-textのP要素に値を渡す
@app.callback(
    Output(component_id=dict(type="show-graph", index=MATCH), component_property="figure"),
    Input(component_id=dict(type="add-dropdown", index=MATCH), component_property="value"),
    Input(component_id=dict(type="add-dropdown2", index=MATCH), component_property="value"),
    prevent_initial_call=False,
)
def update_graph(selected_value, selected_col):
    gap = gapminder[gapminder["country"] == selected_value]
    return px.line(gap, x="year", y=selected_col)

if __name__ == "__main__":
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


### <span style="color: skyblue; ">7.2.3 ALLSMALLER セレクタ</span>

- ALLSMALLER セレクタ
  - ID キー`"type"`の値が共通する ID キー`"index"`の値が小さいすべてのコンポーネントの属性値がコールバック関数内で活用できる


In [ ]:
# gapminder データセットを用いて、ボタンをクリックすると新たにドロップダウンが追加されるアプリケーション
# ドロップダウンでは country 列に含まれる国名が選択できる
# ドロップダウンで選択された国名が出力される
# P要素に、これまでに選択された国名が表示される

from jupyter_dash import JupyterDash
from dash import html, dcc
from dash.dependencies import Input, Output, State, MATCH, ALLSMALLER
import plotly.express as px

gapminder = px.data.gapminder()

app = JupyterDash(__name__)

# レイアウト
def server_layout():
    return html.Div([
        html.Button("PUSH ME", id="add-drop"),
        html.Div(id="show-drop", children=[]),
    ], style=dict(width="80%", margin="2% auto"),)
app.layout = server_layout

# コールバック1
# ボタンを押すたびに
# UI（「ドロップダウン」と「P要素」）が追加される
@app.callback(
    Output(component_id="show-drop", component_property="children"),
    Input(component_id="add-drop", component_property="n_clicks"),
    State(component_id="show-drop", component_property="children"),
    # アプリケーション起動時にコールバックが動作しない
    prevent_initial_call=True,
)
def update_layout(n_clicks, children):
    new_layout = html.Div([
        dcc.Dropdown(
            id=dict(type="add-dropdown", index=n_clicks),
            options=[
                dict(label=c, value=c) for c in gapminder.country.unique()
            ],
            value=gapminder.country.unique()[n_clicks - 1],
        ),
        html.P(id=dict(type="show-text", index=n_clicks)),
    ])
    children.append(new_layout)
    return children

# コールバック2
# UIが追加されたら、
# 選択された値をID=show-textのP要素に値を渡す
@app.callback(
    Output(component_id=dict(type="show-text", index=MATCH), component_property="children"),
    Input(component_id=dict(type="add-dropdown", index=ALLSMALLER), component_property="value"),
    prevent_initial_call=False,
)
def update_graph(selected_values):
    return str(selected_values)

if __name__ == "__main__":
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


In [1]:
# ALLSMALLERセレクタを用いて
# 2つのドロップダウンとグラフが加えられて、
# ドロップダウンで選択した国名と項目をグラフに表示するアプリケーション
# 選択された値より小さい値の項目のグラフを描画

from jupyter_dash import JupyterDash
from dash import html, dcc
from dash.dependencies import Input, Output, State, MATCH, ALLSMALLER
import plotly.express as px

gapminder = px.data.gapminder()

app = JupyterDash(__name__)

# レイアウト
def server_layout():
    return html.Div([
        html.Button("PUSH ME", id="add-drop", n_clicks=0),
        html.Div(id="show-drop", children=[]),
    ])
app.layout = server_layout

# コールバック1
# ボタンを押すたびに
# UI（「ドロップダウン」と「グラフエリア」）が追加される
@app.callback(
    Output(component_id="show-drop", component_property="children"),
    Input(component_id="add-drop", component_property="n_clicks"),
    State(component_id="show-drop", component_property="children"),
    # アプリケーション起動時にコールバックが動作しない
    prevent_initial_call=True,
)
def update_layout(n_clicks, children):
    new_layout = html.Div(
        [
            # 国名表示
            dcc.Dropdown(
                id=dict(type="add-dropdown", index=n_clicks),
                options=[
                    dict(label=c, value=c) for c in gapminder.country.unique()
                ],
                value=gapminder.country.unique()[n_clicks - 1],
            ),
            # 項目表示
            dcc.Dropdown(
                id=dict(type="add-dropdown2", index=n_clicks),
                options=[
                    dict(label=col, value=col) for col in gapminder.columns[3:6]
                ],
                value="lifeExp",
            ),
            # グラフ表示
            dcc.Graph(id=dict(type="show-graph", index=n_clicks)),
        ],
        style=dict(width="50%", display="inline-block")
    )
    children.append(new_layout)
    return children

# コールバック2
# UIが追加されたら、
# 選択された値をID=show-textのP要素に値を渡す
@app.callback(
    Output(component_id=dict(type="show-graph", index=MATCH), component_property="figure"),
    Input(component_id=dict(type="add-dropdown", index=ALLSMALLER), component_property="value"),
    Input(component_id=dict(type="add-dropdown", index=MATCH), component_property="value"),
    Input(component_id=dict(type="add-dropdown2", index=MATCH), component_property="value"),
    prevent_initial_call=False,
)
def update_graph(allsmaller_values, matching_value, selected_col):
    selected_values = allsmaller_values + [matching_value]
    print(f"allsmaller_values: {allsmaller_values}")
    print(f"matching_value: {matching_value}")
    print(f"selected_values: {selected_values}")
    selected_countries = gapminder[gapminder["country"].isin(selected_values)]
    return px.line(selected_countries, x="year", y=selected_col, color="country")

if __name__ == "__main__":
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/
